In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed
0,0,Bluff,NZ,2206939,-46.60,168.33,43.00,86,72,4.00
1,1,Talnakh,RU,1490256,69.49,88.40,60.80,72,2,15.66
2,2,Severo-Kuril'sk,RU,2121385,50.68,156.12,53.31,76,98,8.23
3,3,Saskylakh,RU,2017155,71.92,114.08,43.30,71,90,4.12
4,4,East London,ZA,1006984,-33.02,27.91,62.92,30,0,4.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [4]:
# Heatmap of humidity
locations = city_data_df[["Latitude", "Longitude"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temperature"] < 80) & (city_data_df["Max Temperature"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness (%)"] == 0)].dropna()


narrowed_city_df




,City_ID,City Name,Country,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness (%),Wind Speed
27,27,Vila Velha,BR,3445026,-20.33,-40.29,77.00,78,0,8.05
41,41,Zhanakorgan,KZ,1517323,43.91,67.25,72.50,27,0,5.91
133,133,Mahon,ES,2514301,39.89,4.27,73.40,88,0,8.57
147,147,Marsá Maţrūḩ,EG,352733,31.35,27.25,77.00,78,0,5.82
167,167,Soma,TR,300371,39.19,27.61,73.40,68,0,3.36
199,199,Daşoguz,TM,601734,41.84,59.97,71.60,43,0,6.93
208,208,Salamiyah,SY,164947,35.01,37.05,70.32,69,0,7.83
226,226,Waddān,LY,2209055,29.16,16.14,76.59,49,0,9.86
241,241,Asilah,MA,2557055,35.47,-6.03,71.60,73,0,4.70
253,253,Caravelas,BR,3466980,-17.71,-39.25,71.19,85,0,6.20


In [12]:
city_data_df.keys()


Index(['City_ID', 'City Name', 'Country', 'City ID', 'Latitude', 'Longitude',
       'Max Temperature', 'Humidity', 'Cloudiness (%)', 'Wind Speed'],
      dtype='object')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df [["City Name", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City Name,Country,Latitude,Longitude,Hotel Name
27,Vila Velha,BR,-20.33,-40.29,
41,Zhanakorgan,KZ,43.91,67.25,
133,Mahon,ES,39.89,4.27,
147,Marsá Maţrūḩ,EG,31.35,27.25,
167,Soma,TR,39.19,27.61,
199,Daşoguz,TM,41.84,59.97,
208,Salamiyah,SY,35.01,37.05,
226,Waddān,LY,29.16,16.14,
241,Asilah,MA,35.47,-6.03,
253,Caravelas,BR,-17.71,-39.25,


In [16]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City Name,Country,Latitude,Longitude,Hotel Name
27,Vila Velha,BR,-20.33,-40.29,Hotel Vitória Palace
41,Zhanakorgan,KZ,43.91,67.25,"""Ақпалас"" қонақ үйі"
133,Mahon,ES,39.89,4.27,ARTIEM Capri
147,Marsá Maţrūḩ,EG,31.35,27.25,Beau Site
167,Soma,TR,39.19,27.61,Hotel Labella
199,Daşoguz,TM,41.84,59.97,Hotel Erik
208,Salamiyah,SY,35.01,37.05,Gondolas Forum grandchildren and grandparents
226,Waddān,LY,29.16,16.14,محل الشريف محمد العايب
241,Asilah,MA,35.47,-6.03,Hotel Zelis
253,Caravelas,BR,-17.71,-39.25,Pousada dos Navegantes


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [20]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))